1. Full quantum simulation

First we write down TDSE of the whole system
$$
i \hbar \frac{\partial}{\partial t} \Psi = H \Psi
$$
where the hamiltion can be splited into two parts
$$
H = T_R + H_{el} (r,R)
$$
The second part is electron part of the hamiltonian and now can be presented by the two level model hamiltonian.

While the first part is the nucleus kinetic energy operator $T_R = -\hbar/(2M) \cdot \partial^2/\partial x^2$

In the DVR representation, the 1d space is divided into uniform grid, so the hamiltonian can be writen in a matrix with shape $[ndvr \times nstate, ndvr \times nstate]$
$$
H(x) = \begin{pmatrix} T_R(x) + V_{11}(x) & V_{12}(x) \\ V_{21}(x) & T_R(x) + V_{22}(x)
\end{pmatrix}
$$


the $T_R$ should be written as
$$
T_{i i^{\prime}}=\frac{\hbar^2}{2 M \Delta x^2}(-1)^{i-i^{\prime}}\left\{\begin{array}{ll}
\pi^2 / 3, & i=i^{\prime} \\
\frac{2}{\left(i-i^{\prime}\right)^2}, & i \neq i^{\prime}
\end{array}\right\}
$$
where $x_i = i\Delta x, i = 0, \pm 1, \pm 2$

If we diagonalize the hamiltonian like $H U = UE$, then the TDSE can be solved by
$$
c^{DVR}(t) = exp\left(\frac{Ht}{i\hbar}\right) c^{DVR}(0)
= U exp\left(\frac{Et}{i\hbar}\right) U^\dagger c^{DVR}(0)
$$
where $c^{DVR}$ is the wave function coefficient in the DVR representation.


2. Surface hopping simulation

a. In this method, ions move in a classical trajectory following the Newton's Law
$$
F = M \ddot{x}
$$

And the electrons propagate following the TDSE
$$
i \hbar \frac{\partial}{\partial t} \Psi(r,R,t) = H_{el}(r,R) \Psi(r,R,t) \\
i \hbar \dot{c_i}(t) = \sum_j \left[ E_i^{el}(R)\delta_{ij} - i\hbar d_{ij} \cdot \dot{R} \right] c_j(t)
$$

Solving this equation, we can use the exact diagonazation method stated above. However, since here the hamiltonian is not fixed at different time, the equation can only be solved with discrete time steps.

In the $dt$ time step, electron propagates from state $i$ to state $j$ with probability
$$
P_{i \to j}(t;dt) = -\frac{2}{\hbar} \int_t^{t+dt} \frac{\mathrm{Im}[H_{ij}c_i^* c_j]}{c_i^* c_i} dt
= 2 \int_t^{t+dt} \frac{\mathrm{Re}[d_{ij} \cdot \dot{R} c_i^* c_j]}{c_i^* c_i} dt
$$

Surface hopping describe electron propagation with stochastic trajectories hopping between potential sufaces. The hopping probability is defined by Tully as 
$$
g_{i \to j}(t+dt) = \max(0, P_{i \to j}(t;dt))
$$

After hopping, the ion moment has to be adjusted so that the total energy can to preserved. If the total energy cannot be presevered, then the hop is rejected, which is the so-called frustrated hopping.  
$$
\frac{1}{2}M \dot{R}^2(t) + E_i(t) = \frac{1}{2}M \dot{R}'^2(t) + E_j(t) \\

\dot{R}' = \dot{R} - \frac{\sigma_{ij}}{M} d_{ij} \\

\sigma_{ij} = \frac{B \pm \sqrt{B^2 - 4A (E_i-E_j)}}{2A} \\

A = |d_{ij}|^2 / (2M) \\

B = \dot{R} \cdot d_{ij}
$$
$\sigma_{ij}$ should choose the one with the smaller absoulte value.

Or we can use Morte Carlo algorithm instead of velocity rescaling.
$$
g_{i \to j}(t) \to g_{i \to j}(t) b_{i \to j}(t) \\
b_{i \to j}(t) = \begin{cases} exp\left(-\frac{E_j-E_i}{k_B T}\right) & E_j > E_i \\ 1 & E_j < E_i \end{cases}
$$

In the algorithm, electronic eigen energy, nonadiabatic counpling and force vector should be calculated on every nuclear step.  
The (electronic) eigen energy and wavefunction
$$
H_{el}(x) \psi^{ad}(x) = E_{el}(x) \psi^{ad}(x)
$$

The nonadiabatic coupling vector
$$
d_{ij}(x) = -\frac{\langle\psi_i^{ad} | \nabla H_{el}(x) | \psi_j^{ad} \rangle}
{E_i(x) - E_j(x)}
$$

The force vector 
$$
F_i(x) = - \langle\psi_i^{ad} | \nabla H_{el}(x) | \psi_i^{ad} \rangle
$$

```mermaid
graph TB
   A(ini cond)-->B;
   B(nuclear dynamic)-->C;
   C(wavefunc propagation)-->D;
   D(surface hopping)-->E;
   E(decoherence)-->F(output);
   E-->B
```


velocity verlet algorithm (Swope, 1982)
$$
x(t+dt) = x(t) + v(t)dt + \frac{1}{2}a(t)dt^2 \\
v(t+dt) = v(t) + \frac{1}{2} \left[ a(t) + a(t+dt) \right] dt
$$

3. Initial condition

In the full quantum simulation, the initial wave function is chosen to be a guassian
$$
\psi(x,t=0) = \left(\frac{1}{2\pi \sigma_x^2}\right)^{1/4} 
exp\left(-\frac{(x-x_0)^2}{4\sigma_x^2} + ik_0 x
    \right)
$$

In the surface hoping method, the initial position and moment of the ion should be randomly sampled by the wigner distibution.
$$
P(x,p) = N exp\left(-\frac{(x-x_0)^2}{2\sigma_x^2} \right)
exp\left(-\frac{(p-p_0)^2}{2\sigma_p^2} \right)
$$
where $p_0 = \hbar k_0$ and $\sigma_p = \hbar / (2\sigma_x)$

4. End condition

For full quantum simulation, the charge density will not change when the ion leave the strongly coherent zone.
$$
\rho_i(x) = |c_i(x)|^2 \\
\text{Trans. Lower} \sum_{x>0} |\rho_i(x,t+dt) - \rho_i(x,t)| < \Delta\rho_{thres}, \, i=0
$$
And make sure the wavefunction won't go out of the boundary
$$
\sum_{i, |x|>x_{lim}} |\rho_i(x,t)| < \rho_{thres}
$$

5. Representation transition

basis: $\phi_i^{dia}=1_{\times i}, \phi_i^{ad}=U_{\times i}$, here we assume both coefficients and basis are column vectors and the basis satisfies $\langle\phi_i|\phi_j\rangle=\delta_{ij}, \,\, \sum_i |\phi_i\rangle\langle\phi_i| = 1$
$$
\Psi = \sum_i c_i^{dia} \phi_i^{dia} = \sum_i c_i^{ad} \phi_i^{ad}
$$

then, we have
$$
c_i^{ad} = \sum_j c_j^{dia} \langle\phi_i^{ad} | \phi_j^{dia}\rangle
$$

Under Tully's convention, we use
$$
\hbar = 1, \\
m = 1, M = 2000, \\
\sigma_x = 10/k_0 \\
\mathrm{1 a.u. = 24.188as}

$$

References

[1] J. Chem. Phys. 93, 1061–1071 (1990); https://doi.org/10.1063/1.459170

[2] J. Chem. Phys. 96, 1982–1991 (1992); https://doi.org/10.1063/1.462100

[3] http://staff.ustc.edu.cn/~zqj/posts/Numerical_TDSE/

[4] https://github.com/amber-jain-group-iitb/Exact-QD-Python

[5] https://github.com/smparker/mudslide

* The code here mostly follow the structure of mudslide